In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model

model_name = "microsoft/BioGPT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,   # if needed for custom model code
    use_safetensors=False     # ⛔ disable safetensors fallback
)
# Force CPU if needed
device = torch.device("cpu")
model = model.to(device)

# Create your clinical prompt
prompt = (
    "Example 1:\n"
    "Clinical Summary: A woman presented with fever and neck stiffness after returning from a trip to Africa.\n"
    "Diagnosis: Meningococcal meningitis\n\n"
    "Example 2:\n"
    "Clinical Summary: A man developed sudden chest pain and shortness of breath after climbing stairs.\n"
    "Diagnosis: Pulmonary embolism\n\n"
    "Now solve the following:\n"
    "Clinical Summary: A 22-year-old man was admitted with pain and erythema of the left hand. "
    "He awoke with swelling and bullae in his fingers. He had recently begun practicing taxidermy.\n"
    "Diagnosis:"
)
# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(
        inputs["input_ids"],
        max_length=150,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        top_k=40,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and print
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("=== BioMedLM Output ===")
print(result)


=== BioMedLM Output ===
Example 1: Clinical Summary: A woman presented with fever and neck stiffness after returning from a trip to Africa. Diagnosis: Meningococcal meningitis Example 2: Clinical Summary: A man developed sudden chest pain and shortness of breath after climbing stairs. Diagnosis: Pulmonary embolism Now solve the following: Clinical Summary: A 22-year-old man was admitted with pain and erythema of the left hand. He awoke with swelling and bullae in his fingers. He had recently begun practicing taxidermy. Diagnosis: Pulmonary embolism now solved by ventilation / perfusion scan.
